# Chroma

### 유사도 기반 검색 (Similarity search)

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader(r'E:\Portfolio\Study\hello-langchain\rag_data\history.txt', encoding="utf-8")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)

texts = text_splitter.split_text(data[0].page_content)
texts[0]

e:\Portfolio\Study\hello-langchain\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.\n\n고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.'

**Chroma 벡터 저장소 생성**

- `from_texts` 메소드를 사용하여 분할된 텍스트들을 임베딩하고, 이 임베딩을 Chroma 벡터 저장소에 저장합니다.

- 저장소는 `collection_name`으로 구분되며, 여기서는 'history'라는 이름을 사용합니다.

- 저장된 데이터는 `./db/chromadb` 디렉토리에 저장됩니다.

- `collection_metadata`에서 `'hnsw:space': 'cosine'`을 설정하여 유사도 계산에 코사인 유사도를 사용합니다.

In [2]:
embeddings_model = OpenAIEmbeddings()
db = Chroma.from_texts(
    texts,
    embedding=embeddings_model,
    collection_name='history',
    persist_directory='./db/chromadb',
    collection_metadata={'hnsw:space': 'cosine'}
)

db

In [3]:
query = "누가 한글을 창제했나요?"
docs = db.similarity_search(query)
print(docs[0].page_content)

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


### MMR (Maximum marginal relevance search)
- MMR 검색 방식은 **유사성과 다양성의 균형**을 맞추어 검색 결과의 품질을 향상시키는 알고리즘입니다. 

- 검색 쿼리에 대한 문서들의 관련성을 최대화하는 동시에, 검색된 문서들 사이의 중복성을 최소화하여, 사용자에게 다양하고 풍부한 정보를 제공하는 것을 목표로 합니다.

**주요 매개변수**

- `query`: 사용자로부터 입력받은 검색 쿼리입니다.

- `k`: 최종적으로 선택할 문서의 수입니다. 이 매개변수는 반환할 문서의 총 개수를 결정합니다.

- `fetch_k`: MMR 알고리즘을 수행할 때 고려할 상위 문서의 수입니다. 이는 초기 후보 문서 집합의 크기를 의미하며, 이 중에서 MMR에 의해 최종 문서가 k개 만큼 선택됩니다.

- `lambda_mult`: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절합니다. 
    - 1은 유사성만 고려 

    - 0은 다양성만을 최대화

In [5]:
# 유사도 기반 검색

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = PyMuPDFLoader(r'E:\Portfolio\Study\hello-langchain\rag_data\300720_한일시멘트_2023.pdf')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

228

In [6]:
embeddings_model = OpenAIEmbeddings()
db2 = Chroma.from_documents(
    documents, 
    embeddings_model,
    collection_name = 'esg',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db2

In [ ]:
query = "한일시멘트의 주요 approach를 알려줘"
docs = db2.similarity_search(query)
print(len(docs))
print(docs[0].page_content)  # 가장 유사한 문서

4
CEO MESSAGE
한일시멘트 · 한일현대시멘트
장기적인 안목과 적극적인 투자로 
기후변화를 위기가 아닌 기회로 전환시키는 
한일시멘트. 한일현대시멘트가 되겠습니다. 
존경하는 한일 이해관계자 여러분, 안녕하십니까.
한일의 지속적인 성장과 발전에 변함없는 성원과 관심을 기울여 주신 주주, 고객,  
협력사, 임직원분들께 감사의 인사를 드립니다.
지난해 시멘트 업계는 유연탄 등 글로벌 원자재 가격 폭등으로 수익성이 악화되었고, 
화물연대 파업, 건설경기 침체 등 대내외적으로 어려운 시기를 보내야 했습니다. 이에 
더해 폭염 · 폭우 · 가뭄 등 전 세계적인 기상 이변은 자연재해뿐만 아니라 생태계 파괴, 
식량위기 등 다양한 문제들을 야기하고 있습니다. 친환경의 시대를 넘어 인류 생존을 
위한 필(必)환경의 시대임을 절감케하는 변화가 현실이 된 것입니다.
한일은 급변하는 대내외 환경 속에서도 장기적인 안목을 가지고 환경 · 사회 · 지배
구조를 중심으로 지속가능경영을 추구하는 ESG 실천에 역량을 집중하고 있습니다. 
특히 온실가스 및 대기오염물질 감축과 같은 환경 이슈는 위기를 넘어 기회로 전환
할 수 있도록 적극적인 대응에 나서고 있습니다. 
한일은 ‘2030 온실가스 30% 감축, 2050 Net Zero’ 달성을 위해서 저탄소 친환경 
생산공정 구축에 필요한 기술 개발 및 설비투자를 지속하고 있습니다. 2025년까지 
단계적으로 시멘트 공장의 소성로를 개조하여 화석연료 의존도를 낮추고, 순환연료
의 비중을 확대하는 한편, 석회석 등 천연원료를 순환원료로 대체하여 자원순환형 
사회 구축에 기여하도록 하겠습니다. 또한 시멘트 소성공정 중에 발생되는 폐열을 
회수하여 전기를 생산하는 ECO 발전을 확대하여 전력 자립도를 높이겠습니다.
또한 환경을 둘러싼 규제가 강화되고 기업의 오염물질 관리가 더욱 중요해지는 사회
변화에 발맞추어 엄격한 내부기준으로 오염물질 배출을 최소화하고, 생산활동의 환경
영향을 지속적으로 관리하여 사회의 요구에 부합하도록 하겠습니다.


In [8]:
print(docs[-1].page_content)  # 가장 유사도가 낮은 문서

한일은 중대성평가 결과 도출된 핵심 이슈에 대해서 대응 전략을 세우고 세부 추진과
제를 실천하고 있습니다. 각 이슈의 성과는 KPI로 관리되며, 경영진의 성과평가에 이
를 반영하여 ESG경영을 독려하고 있습니다. 
환경
(E)
사회
(S)
핵심 이슈 관리
핵심이슈
주요 영향
전략 과제
세부 추진 과제
 
KPI
2030 목표
기후변화 대응 전략
기후변화는 이제 우리사회가 다 함께  
풀어야할 공통의 숙제로  
인식되고 있습니다. 특히 시멘트 산업은  
온실가스 다배출 업종으로  
기후위기 극복과 탄소중립 달성이라는  
시대적 과제에 직면해 있습니다.  
기후변화 대응 전략 및 실행 로드맵 수립
화석연료 사용 최소화
석회석 및 클링커 대체
신재생 에너지 대체
온실가스 배출량
온실가스 배출량
오염물질 관리
시멘트 생산과정의 특성을 활용하여  
폐합성수지를 재생연료로 사용하고  
다양한 재생원료를 사용하는 것은  
사회적 가치를 창출하는 일입니다.  
그러나 순환자원의 사용 효율성을 높이기  
위한 지속적인 기술 개발과 시설 개선이  
요구되며, 이 과정에서 발생하는 질소산
화물 등 또다른 환경문제를 일으킬 위험
성을 내포하고 있습니다.  
오염물질 배출 감축 로드맵 수립
오염물질 배출 관리 강화
오염물질 저감 설비 구축
질소산화물 배출량
질소산화물 배출량
친환경 사업 개발
기후변화에 대한 사회적 제재는 기업에게  
위기 요소가 되지만, 반대로 친환경,  
저탄소 기술의 발달로 지속가능 산업이라
는 기회를 만들기도 합니다. 많은 기업들
이 친환경을 테마로 미래세대의 핵심  
성장동력을 찾고 있습니다. 적극적인 대
응은 신규사업 기회 뿐만 아니라 기업의 
브랜드 가치, 임직원의 자긍심 고취에도 
긍정적인 영향을 미칠 수 있습니다.  
시멘트 생산 연계 친환경 신규사업 개발
친환경 제품 개발 및 인증 확대
친환경 사업 밸류체인 확대
신규사업 매출액
친환경 매출 비중
안전하고 건강한 작업환경 구축


In [9]:
# MMR 검색
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

4
CEO MESSAGE
한일시멘트 · 한일현대시멘트
장기적인 안목과 적극적인 투자로 
기후변화를 위기가 아닌 기회로 전환시키는 
한일시멘트. 한일현대시멘트가 되겠습니다. 
존경하는 한일 이해관계자 여러분, 안녕하십니까.
한일의 지속적인 성장과 발전에 변함없는 성원과 관심을 기울여 주신 주주, 고객,  
협력사, 임직원분들께 감사의 인사를 드립니다.
지난해 시멘트 업계는 유연탄 등 글로벌 원자재 가격 폭등으로 수익성이 악화되었고, 
화물연대 파업, 건설경기 침체 등 대내외적으로 어려운 시기를 보내야 했습니다. 이에 
더해 폭염 · 폭우 · 가뭄 등 전 세계적인 기상 이변은 자연재해뿐만 아니라 생태계 파괴, 
식량위기 등 다양한 문제들을 야기하고 있습니다. 친환경의 시대를 넘어 인류 생존을 
위한 필(必)환경의 시대임을 절감케하는 변화가 현실이 된 것입니다.
한일은 급변하는 대내외 환경 속에서도 장기적인 안목을 가지고 환경 · 사회 · 지배
구조를 중심으로 지속가능경영을 추구하는 ESG 실천에 역량을 집중하고 있습니다. 
특히 온실가스 및 대기오염물질 감축과 같은 환경 이슈는 위기를 넘어 기회로 전환
할 수 있도록 적극적인 대응에 나서고 있습니다. 
한일은 ‘2030 온실가스 30% 감축, 2050 Net Zero’ 달성을 위해서 저탄소 친환경 
생산공정 구축에 필요한 기술 개발 및 설비투자를 지속하고 있습니다. 2025년까지 
단계적으로 시멘트 공장의 소성로를 개조하여 화석연료 의존도를 낮추고, 순환연료
의 비중을 확대하는 한편, 석회석 등 천연원료를 순환원료로 대체하여 자원순환형 
사회 구축에 기여하도록 하겠습니다. 또한 시멘트 소성공정 중에 발생되는 폐열을 
회수하여 전기를 생산하는 ECO 발전을 확대하여 전력 자립도를 높이겠습니다.
또한 환경을 둘러싼 규제가 강화되고 기업의 오염물질 관리가 더욱 중요해지는 사회
변화에 발맞추어 엄격한 내부기준으로 오염물질 배출을 최소화하고, 생산활동의 환경
영향을 지속적으로 관리하여 사회의 요구에 부합하도록 하겠습니다.


In [10]:
print(mmr_docs[-1].page_content)

위해 다양한 형태와 수준의 이해관계자 커뮤니케이션 채널을 개발하여 
운영하고 있습니다. 검증팀은 이 과정에서 누락된 주요 이해관계자 그
룹을 발견할 수 없었으며, 그들의 견해 및 기대 사항이 적절히 조직의 
전략에 반영되도록 노력하고 있음을 확인하였습니다.
중요성 원칙
한일은 조직의 지속가능성 성과에 미치는 주요 이슈들의 중요성을 고
유의 평가 프로세스를 통해 결정하고 있으며, 검증팀은 이 프로세스에
서 누락된 중요한 이슈를 발견하지 못하였습니다.
대응성 원칙
한일은 도출된 주요 이슈의 우선순위를 정하여 활동성과와 대응사례 그
리고 향후 계획을 포괄적이고 가능한 균형 잡힌 방식으로 보고하고 있
으며, 검증팀은 한일의 대응 활동들이 보고서에 부적절하게 기재되었
다는 증거를 발견하지 못하였습니다.
영향성 원칙
한일은 중대성 평가를 통해 파악된 주요 주제들의 직간접적인 영향을 
파악하여 모니터링하고 있으며 가능한 범위 내에서 해당 영향을 정량
화된 형태로 보고하고 있음을 확인하였습니다.
특정 지속가능성 성과정보의 신뢰성 및 품질
검증팀은 AA1000AP(2018) 원칙 준수 여부에 더해 지속가능성 성과
와 관련된 경제, 환경, 사회 성과 정보에 대한 신뢰성 검증을 실시하였
습니다. 해당 정보 및 데이터의 검증을 위해 담당자와 인터뷰를 실시하
였으며, 데이터 샘플링 및 근거 문서 그리고 외부 출처 및 공공 데이터
베이스를 통해 신뢰할 수 있는 데이터임을 확인하였습니다. 검증팀은 
지속가능성 성과 정보에서 의도적 오류나 잘못된 기술을 발견하지 못
하였습니다.
적격성 및 독립성
한국경영인증원은 ISO/IEC 17021·2015(경영시스템의 심사 및 인증
을 제공하는 기관에 대한 요구사항)에 따라 문서화된 방침과 절차를 포
함한 포괄적인 품질관리시스템을 유지하고 있습니다. 검증팀은 지속가
능성 전문가들로 구성되어 있으며, 제3자 검증서비스를 제공하는 업무 
이외에 한일의 사업활동에 영리를 목적으로 하는 어떠한 이해관계도 맺
지 않은 독립성을 유지하고 있습니다.
2023.0